In [1]:
%load_ext lab_black

In [2]:
import os

os.chdir("..")

In [3]:
import pandas as pd
from pipelines.pipelines_config import *

In [4]:
df = pd.read_pickle(train_data_path)

In [5]:
df

,doc_id,bow_dict,label
0,8018,"{'afternoon': 1, 'attached': 1, 'board': 1, 'b...",enron
1,126257,"{'accompli': 1, 'accomplished': 1, 'account': ...",nytimes
2,278701,"{'able': 1, 'ace': 1, 'advantage': 1, 'adverti...",nytimes
3,97518,"{'able': 1, 'abortion': 5, 'abundance': 1, 'ab...",nytimes
4,157394,"{'according': 1, 'actually': 1, 'address': 1, ...",nytimes
...,...,...,...
24276,35833,"{'assistant': 1, 'awaiting': 1, 'brian': 1, 'p...",enron
24277,12748,"{'add': 1, 'afternoon': 2, 'american': 1, 'att...",enron
24278,26,"{'addition': 1, 'administration': 1, 'alive': ...",kos
24279,120,"{'add': 1, 'analysis': 1, 'answer': 1, 'attent...",kos


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [7]:
def dict_to_string(dic):
    items = [[item[0]] * item[1] for item in dic.items()]
    return " ".join([str(item) for l in items for item in l if item])

In [8]:
tfidf = TfidfVectorizer(use_idf=False, norm=None)
df["string"] = df["bow_dict"].apply(dict_to_string)
tfidf.fit(df["string"])

TfidfVectorizer(norm=None, use_idf=False)

In [9]:
transformed = tfidf.transform(df["string"])

In [20]:
import numpy as np

In [25]:
inv_vocab = {item[1]: item[0] for item in tfidf.vocabulary_.items()}

In [46]:
too_rare_mask = np.array((transformed > 0).sum(axis=0) <= 2)[0]
too_rare_indices = np.where(too_rare_mask)
too_rare_tokens = np.vectorize(inv_vocab.get)(too_rare_indices)[0]

In [48]:
from funcy import select_keys

In [49]:
def filter_too_rare(bow_dict: dict, too_rare_tokens: np.ndarray):
    return select_keys(lambda key: key not in too_rare_tokens, bow_dict)

In [50]:
import swifter

In [53]:
temp = df.sample(frac=0.01)

In [54]:
temp["bow_dict"].swifter.apply(filter_too_rare, too_rare_tokens=too_rare_tokens)

Pandas Apply:   0%|          | 0/243 [00:00<?, ?it/s]

2732     {'ability': 1, 'admit': 1, 'amazing': 1, 'anyw...
18752    {'arise': 1, 'characterized': 2, 'clinical': 1...
4372     {'able': 1, 'air': 1, 'annual': 1, 'appreciate...
9610     {'arising': 1, 'bacteria': 1, 'bacterial': 1, ...
3918     {'able': 1, 'access': 1, 'according': 1, 'addi...
                               ...                        
13135    {'closed': 8, 'database': 8, 'date': 1, 'downl...
17484    {'able': 1, 'april': 1, 'attend': 1, 'bacon': ...
2850     {'action': 1, 'add': 3, 'addition': 1, 'admini...
18922    {'accident': 3, 'adjoining': 2, 'agencies': 1,...
21097    {'account': 2, 'aerobic/anaerobic': 2, 'air-wa...
Name: bow_dict, Length: 243, dtype: object